# Tensorboard

1. Present methods of how to use
2. Present full code usage

### Resources
- [My Youtube guy](https://www.youtube.com/watch?time_continue=297&v=3bownM3L5zM) - 5 minutes expalantion- Very useful
- [Tensorboard in 5 minutes](https://medium.com/@anthony_sarkis/tensorboard-quick-start-in-5-minutes-e3ec69f673af) - quick referance 
- [Tensorboard Notebook example](https://github.com/swirlingsand/tensorboard-rnn-example/blob/master/Tensorboard%20example.ipynb) Recommanded
- [Datacamp tutorial](https://www.datacamp.com/community/tutorials/tensorboard-tutorial) - Thorough TensorBoard tutorial 
- [Tensorboard page](https://medium.com/@anthony_sarkis/tensorboard-quick-start-in-5-minutes-e3ec69f673af)
- [Oficial Tensorboard Git](https://github.com/tensorflow/tensorboard) - Very Useful, with many commands and expalantios

### How to Use
There are two ways to add data to tensorboard

##### 1. Adding a variable to tf.summary
There are multiple actions TF knows to do on objects. 

List of actions:
https://www.tensorflow.org/api_guides/python/summary

#### Summary Function

1. tf.summary.image: 

##### 2. Save variabes During Training
Stages:
- In your tf.Session() define a writer.
- In your training loop, define a merge with tf.summary.merge_all().
- In sess.run() pass merge (a Tensor), get back a summary.
- Add summary to your writer. Done.

Next steps:
- Start the training
- In terminal run Tensorboard:  tensorboard --logdir logs/1
- In browser go to URL provided: http://localhost:6006/

### 3. Next Step - Add Name Scopes
It is assential to add name scope to your function and quite eay to do it after the code. <br>
 Just add tf.name_scope() to the functions

# Tensorboard Example

#### Set helper functions

In [3]:
import tensorflow as tf

def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name=name)

def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    with tf.name_scope("layer1"):
        X = tf.nn.dropout(X, p_keep_input)
        h = tf.nn.relu(tf.matmul(X, w_h))
    with tf.name_scope("layer2"):
        h = tf.nn.dropout(h, p_keep_hidden)
        h2 = tf.nn.relu(tf.matmul(h, w_h2))
    with tf.name_scope("layer3"):
        h2 = tf.nn.dropout(h2, p_keep_hidden)
    return tf.matmul(h2, w_o)

In [5]:
# Step 1 - Get input data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
# Step 2 - Create input and output placeholders
X = tf.placeholder("float", [None, 784] , name="X")
Y = tf.placeholder("float", [None, 20] , name="Y")

# Step 3 - Initiliaze weights:
w_h = init_weights([784, 625], "w_h")
w_h2 = init_weights([625, 625], "w_h2")
w_o = init_weights([625, 10], "w_o")

In [7]:
# Step 4 - Add histogram summaries:
tf.summary.histogram("w_h_sum", w_h)
tf.summary.histogram("w_h2_sum", w_h2)
tf.summary.histogram("w_o_sum", w_o)

<tf.Tensor 'w_o_sum:0' shape=() dtype=string>

In [8]:
# Step 5 - Add dropout to input and hidden layers:
p_keep_input = tf.placeholder("float", name="p_keep_input")
p_keep_hidden = tf.placeholder("float", name="p_keep_hidden")

In [9]:
# Step 6 - Create model:
p_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

In [11]:
# Step 7 - Create cost function:
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=p_x,labels=Y))
    train_op = tf.train.RMSPropOptimizer(learning_rate=0.001, decay=0.9).minimize(cost)
    # Add scalar summary for cost tensor:
    tf.summary.scalar("cost", cost)

In [13]:
# Step 8 - Measure Acuracy
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.arg_max(Y,1), tf.arg_max(p_x,1))
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float")) #Cast boolean to float to get average
    tf.summary.scalar("accuracy", acc_op)

In [1]:
# Step 9 - Create Session

# Merge all the summaries and write them out to
log_dir ='/tmp/tensorflow/mnist/logs/mnist_with_summaries' 

with tf.Session() as sess:
    # Step 10 - Create a log writer. Run 'tensorboard --logdir="log_dir"'
    writer = tf.summary.FileWriter(log_dir, sess.graph)
    merged = tf.summary.merge_all()
    
    # Step 11 - Initializa al variables:
    tf.global_variables_initializer().run()

    # Step 12 - Train he model
    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
        #Note - the range thing is a nice way to get [0,128,256,....]
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end], p_keep_input: 0.8, p_keep_hidden: 0.5})
        summary, acc = sess.run([merged, acc_op], feed_dict={X: trX[start:end], Y: trY[start:end], p_keep_input: 1, p_keep_hidden: 1})
        writer.add_summary(summary, i)
        print(i, acc)

NameError: name 'tf' is not defined

## Tensorboard on PyTorch

You will need to install tensorboard-pytorch and tensorboardX:    pip intall tensorboard-pytorch tensorboardX 
<br>Check out [this](https://medium.com/@dexterhuang/tensorboard-for-pytorch-201a228533c5) source

In [6]:
#Usage example:
try:
    from tensorboardX import SummaryWriter
    is_tensorboard_available = True
except Exception:
    is_tensorboard_available = False
# Write you code
writer = SummaryWriter('tensorboard_outputs') if is_tensorboard_available else None

# Inspect TF-Record
If you would like to inspect the content of a TF-Record. 
[Source](https://stackoverflow.com/questions/42394585/how-to-inspect-a-tensorflow-tfrecord-file)

In [2]:
import tensorflow as tf

for example in tf.python_io.tf_record_iterator("path_to.tfrecord"):
    result = tf.train.Example.FromString(example)
    
# OR
from google.protobuf.json_format import MessageToJson
...
jsonMessage = MessageToJson(tf.train.Example.FromString(example))

NotFoundError: path_to.tfrecord; No such file or directory

# TF on Cloud

1. Run tensorboard on the coud 
        tensorboard --logdir <path to file or dir containing the even logs> --port 6006  #port 6006 is the defualt port anyway
    
2. Once you run the TensoerBoard on the cloud on can connect to it via:
        ssh -i ~/.ssh/shimkin_cloud_key -L 6006:127.0.0.1:6006 blink@35.231.131.217
   In case you need to debug you can run:
        ssh -L 6006:127.0.0.1:6006 blink@35.237.122.77 -N -v -v
3. Now just open a browser and type: http://127.0.0.1:6006

## Debuggings:

Error output:  (in the cloud)<br>
    return _setlocale(category, locale)<br>
locale.Error: unsupported locale setting<br>
[Solution](https://github.com/tensorflow/tensorboard/issues/1323):<br>
export LC_ALL="en_US.UTF-8" <br>
export LC_CTYPE="en_US.UTF-8"<br>
sudo dpkg-reconfigure locales<br>
OR:<br>
export LC_ALL=C
